In [1]:
# -*- coding: utf-8 -*-
import pyodbc
import pandas as pd
import numpy as np
import os.path
import uuid
#from datetime import time
%matplotlib inline



In [6]:
um = pd.read_excel('52nd Heckerling Institute USB.xlsx', sheetname='Sessions', header=0,)
speak = pd.read_excel('52nd Heckerling Institute USB.xlsx', sheetname='Speakers', header=0 )
#print um.dtypes
speak = speak.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
speak['bio'] = speak.SpeakerBio.str.replace('.pdf', '.html')
um = um.fillna('')
speak = speak.fillna('')
um['speakers'] = um[um.columns[7:13]].applymap(lambda x: [x] if (x != '') else []).sum(1).tolist()
uid = []
for row in um['ProgramTitle']:
    uid.append(uuid.uuid4())
um['UID'] = uid
um['UID'] = um['UID'].astype(str)
cols = um[um.columns[7:13]]
um.drop(cols, axis=1, inplace = True)
#um = um.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [10]:
# read in biographies from html.
bx = []
for sp in speak.bio:
    if os.path.isfile(sp):
        f = open(str(sp),'r')
        message = f.read()
        #print(message)
        f.close()
        bx.append(message)
    else:
        bx.append('')
speak['biotext'] = bx  

# Concatenate speaker full name to "speaker" column for matching purposes.
cat = []
for row in speak.itertuples():
    if row.SpeakerMiddle:
        cat.append(row.SpeakerFirstName + ' ' + row.SpeakerMiddle + ' ' + row.SpeakerLastName)
    else:
        cat.append(row.SpeakerFirstName  + ' ' + row.SpeakerLastName)
        
speak['speaker'] = cat


In [7]:
um.to_json('umtest.json',orient='records',date_format='iso' )

In [ ]:
speak.to_json('speakers.json',orient='records' )

In [16]:
speak[50:52]

,SpeakerLastName,SpeakerFirstName,SpeakerMiddle,SpeakerBio,bio,biotext,speaker
50,Pharies,S.,Andrew,Pharies_Bio2018.pdf,Pharies_Bio2018.html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01//...",S. Andrew Pharies
51,"Pluth, Jr.",Robert,R.,Pluth_Bio2018.pdf,Pluth_Bio2018.html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01//...","Robert R. Pluth, Jr."


In [ ]:
um.head()